In [1]:
import json
from datasets import load_dataset

D:\PyCharmProjects\LLaMA-Factory\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 1. Create inverse guanaco dataset

In [2]:
hf_token = r"hf_"
dataset = load_dataset('timdettmers/openassistant-guanaco', token=hf_token)
dataset

Repo card metadata block was not found. Setting CardData to empty.


DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 9846
    })
    test: Dataset({
        features: ['text'],
        num_rows: 518
    })
})

In [3]:
def parse_conversation(text_dict):
    text = text_dict['text']
    # Find the positions of the markers
    human_start = text.find("### Human: ")
    assistant_start = text.find("### Assistant: ")
    next_human_start = text.find("### Human: ", assistant_start)
    
    # Extract the first human query
    if next_human_start != -1:
        human_text = text[human_start + len("### Human: "):assistant_start].strip()
    else:
        human_text = text[human_start + len("### Human: "):assistant_start].strip()
    
    # Extract the first assistant response
    if next_human_start != -1:
        assistant_text = text[assistant_start + len("### Assistant: "):next_human_start].strip()
    else:
        assistant_text = text[assistant_start + len("### Assistant: "):].strip()
    
    # Create and return the dictionary
    return {
        "human": human_text,
        "assistant": assistant_text
    }


In [4]:
parsed_dataset = [parse_conversation(text) for text in dataset['train']]
parsed_dataset[0]

{'human': 'Can you write a short introduction about the relevance of the term "monopsony" in economics? Please use examples related to potential monopsonies in the labour market and cite relevant research.',
 'assistant': '"Monopsony" refers to a market structure where there is only one buyer for a particular good or service. In economics, this term is particularly relevant in the labor market, where a monopsony employer has significant power over the wages and working conditions of their employees. The presence of a monopsony can result in lower wages and reduced employment opportunities for workers, as the employer has little incentive to increase wages or provide better working conditions.\n\nRecent research has identified potential monopsonies in industries such as retail and fast food, where a few large companies control a significant portion of the market (Bivens & Mishel, 2013). In these industries, workers often face low wages, limited benefits, and reduced bargaining power, le

In [5]:
instruction_data = []
for sample in parsed_dataset:
    instruction_data.append({
        "instruction": "Your task is to infer and generate the user’s original input question that could have led to the following Assistant response.",
        "input": sample["assistant"],
        "output": sample["human"]
    })
instruction_data[0]

{'instruction': 'Your task is to infer and generate the user’s original input question that could have led to the following Assistant response.',
 'input': '"Monopsony" refers to a market structure where there is only one buyer for a particular good or service. In economics, this term is particularly relevant in the labor market, where a monopsony employer has significant power over the wages and working conditions of their employees. The presence of a monopsony can result in lower wages and reduced employment opportunities for workers, as the employer has little incentive to increase wages or provide better working conditions.\n\nRecent research has identified potential monopsonies in industries such as retail and fast food, where a few large companies control a significant portion of the market (Bivens & Mishel, 2013). In these industries, workers often face low wages, limited benefits, and reduced bargaining power, leading to a situation where they are dependent on the employer for 

In [12]:
with open("data/guanaco_inverse.json", "w") as f:
    json.dump(instruction_data, f, indent=4)

# 2. Create Single turn LIMA dataset

In [6]:
dataset = load_dataset('GAIR/lima', token=hf_token)

length_distribution = []
for sample in dataset['train']:
    conversation = sample['conversations']
    length_distribution.append(len(conversation))
    if len(conversation) > 2:
        for l in conversation:
            print(l)
            print("*" * 50)
        break

Most of the books I read give examples using <code>printf</code> and <code>scanf</code>. At some point the students know perfectly how to use these two functions but they don't know about <code>stdin</code>, <code>stdout</code> and <code>argv</code>. 
To me and according to many programming principles (e.g. KISS) a program should not interrupt the execution for prompting the user. Instead, and this is a much clever approach, the developer should learn to use the power of pipes and use the options and the arguments. 
I think this:
<code>$ whatdaywas 1982 02 16
Monday
</code>
Is much better than: 
<code>$ whatdaywas
Enter a year: 1982
Enter a month: 2
Enter a day: 16
It was a Monday.
</code>
Is there any rationale behind this pedagogical approach?
**************************************************
When tutoring young CS students, user input is actually a tricky concept. If it were not for user input, the results of any program could be pre-compiled into nothing but its output. You get on

In [7]:
def convert_to_single_turn_conversations(dataset):
    """
    Converts the dataset into a list of single-turn conversations.
    
    Args:
        dataset: The dataset containing conversation samples
        
    Returns:
        List of dictionaries, each with 'human' and 'assistant' keys
    """
    single_turn_conversations = []
    
    # Iterate through the train split of the dataset
    for sample in dataset['train']:
        conversation = sample['conversations']
        
        # Check if this is a single-turn conversation (2 entries)
        if len(conversation) == 2:
            # First entry is human/user, second is assistant/model
            single_turn_conversations.append({
                "human": conversation[0],
                "assistant": conversation[1]
            })
    
    return single_turn_conversations

In [ ]:
single_turn_conversations = convert_to_single_turn_conversations(dataset)
with open("data/lima_single_turn.json", "w") as f:
    json.dump(single_turn_conversations, f, indent=4)